In [1]:
import geopandas as gpd
import ast
import os

# Load data
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
crit_xing_path = os.path.join(base_dir, "critical_crossings.shp")

# Read the shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)

# Convert string representations of lists to actual lists if needed
if isinstance(crossings['FRM_LEG'].iloc[0], str):
    crossings['FRM_LEG'] = crossings['FRM_LEG'].apply(ast.literal_eval)
if isinstance(crossings['TO_LEG'].iloc[0], str):
    crossings['TO_LEG'] = crossings['TO_LEG'].apply(ast.literal_eval)

# Convert all IDs to integers
roads['unique_id'] = roads['unique_id'].astype(int)
crossings['FRM_LEG'] = crossings['FRM_LEG'].apply(lambda x: [int(i) for i in x] if x else [])
crossings['TO_LEG'] = crossings['TO_LEG'].apply(lambda x: [int(i) for i in x] if x else [])

# Create LTS lookup
lts_lookup = roads.set_index('unique_id')['LTS_2025'].to_dict()

# Add new columns
crossings['LTS_FRM'] = crossings['FRM_LEG'].apply(lambda x: lts_lookup.get(x[0]) if x else None)
crossings['LTS_TO'] = crossings['TO_LEG'].apply(lambda x: lts_lookup.get(x[0]) if x else None)

In [2]:
crossings[['JUNC_ID', 'FRM_LEG', 'TO_LEG', 'LTS_FRM', 'LTS_TO']].head()

,JUNC_ID,FRM_LEG,TO_LEG,LTS_FRM,LTS_TO
0,5.0,[1830],[13],3,1
1,5.0,[1827],"[36281, 3383]",2,2
2,5.0,[13],[3382],1,2
3,5.0,"[36281, 3383]",[1830],2,3
4,5.0,[3382],[1827],2,2


In [3]:
print(crossings['LTS_FRM'].value_counts())
print(crossings['LTS_TO'].value_counts())

LTS_FRM
1    26670
3     7832
2     4992
4     1550
6      859
5       61
Name: count, dtype: int64
LTS_TO
1    26670
3     7832
2     4992
4     1550
6      859
5       61
Name: count, dtype: int64


In [4]:
crossings.to_file(crossings_path)

In [5]:
import pandas as pd

# Ensure numeric types
for col in ['xLTS', 'LTS_FRM', 'LTS_TO']:
    crossings[col] = pd.to_numeric(crossings[col], errors='coerce')

# Calculate max LTS
crossings['max_LTS'] = crossings[['LTS_FRM', 'LTS_TO']].max(axis=1, skipna=False)

# Define the critical conditions
condition1 = (crossings['xLTS'] == 2) & (crossings['max_LTS'] == 1)
condition2 = (crossings['xLTS'] == 3) & (crossings['max_LTS'] == 1)
condition3 = (crossings['xLTS'] == 3) & (crossings['max_LTS'] == 2)

# Create the subset (all columns, only critical rows)
crit_xing = crossings[condition1 | condition2 | condition3].copy()

# Verification counts using the original conditions
print(f"Critical crossings count: {len(crit_xing)}")
print("\nBreakdown:")
print(f"xLTS=2 & max=1: {crit_xing.loc[condition1[crit_xing.index]].shape[0]}")
print(f"xLTS=3 & max=1: {crit_xing.loc[condition2[crit_xing.index]].shape[0]}")
print(f"xLTS=3 & max=2: {crit_xing.loc[condition3[crit_xing.index]].shape[0]}")

# Save to new shapefile
crit_xing.to_file(crit_xing_path, driver='ESRI Shapefile')

Critical crossings count: 396

Breakdown:
xLTS=2 & max=1: 291
xLTS=3 & max=1: 67
xLTS=3 & max=2: 38


In [6]:
crossings[['JUNC_ID', 'FRM_LEG', 'TO_LEG', 'LTS_FRM', 'LTS_TO', 'xLTS', 'max_LTS']].head()

,JUNC_ID,FRM_LEG,TO_LEG,LTS_FRM,LTS_TO,xLTS,max_LTS
0,5.0,[1830],[13],3,1,2,3
1,5.0,[1827],"[36281, 3383]",2,2,1,2
2,5.0,[13],[3382],1,2,2,2
3,5.0,"[36281, 3383]",[1830],2,3,2,3
4,5.0,[3382],[1827],2,2,1,2
